In [1]:
## Univariate, Single-Step Model

In [18]:
import numpy as np
import pandas as pd
import xgboost as xgb
import sklearn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [19]:
import tensorflow as tf

In [20]:
#SPEI - 1 month, 6 month models (LSTM)
#SPEI with humidity, temperature models - 1 month (LSTM)
#ConvLSTM if time permits

In [21]:
#SPEI 1 month model

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
df = pd.read_csv("/content/drive/MyDrive/data_senegal/senegal_weather_spei_1970_2020_025x025_v3.csv")

In [24]:
df = df.dropna()
df = df.reset_index(drop=True)

In [25]:
spei_values = df["spei_1mon"]

In [26]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
        
    return np.array(X), np.array(y)

In [27]:
n_steps = 12
X, y = split_sequence(spei_values, n_steps)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [29]:
scaler = StandardScaler()
X_train = scaler.fit_transform(np.array(X_train))
scaler = StandardScaler()
X_test = scaler.fit_transform(np.array(X_test))
scaler = StandardScaler()
y_train = scaler.fit_transform(y_train.reshape(-1,1))
scaler = StandardScaler()
y_test = scaler.fit_transform(y_test.reshape(-1,1))

In [30]:
n_features = 1 #Since it's a univariate model

model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [31]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))

In [32]:
with tf.device('/device:GPU:0'):
  model.fit(X_train, y_train, epochs=60, batch_size=64)

Epoch 1/60
2309/2309 [==============================] - 97s 41ms/step - loss: 0.7477
Epoch 2/60
2309/2309 [==============================] - 95s 41ms/step - loss: 0.7165
Epoch 3/60
2309/2309 [==============================] - 93s 40ms/step - loss: 0.6844
Epoch 4/60
2309/2309 [==============================] - 97s 42ms/step - loss: 0.6319
Epoch 5/60
2309/2309 [==============================] - 93s 40ms/step - loss: 0.5592
Epoch 6/60
2309/2309 [==============================] - 96s 42ms/step - loss: 0.5044
Epoch 7/60
2309/2309 [==============================] - 92s 40ms/step - loss: 0.4533
Epoch 8/60
2309/2309 [==============================] - 93s 40ms/step - loss: 0.3993
Epoch 9/60
2309/2309 [==============================] - 98s 43ms/step - loss: 0.3676
Epoch 10/60
2309/2309 [==============================] - 94s 41ms/step - loss: 0.3325
Epoch 11/60
2309/2309 [==============================] - 96s 42ms/step - loss: 0.3072
Epoch 12/60
2309/2309 [==============================] - 94s 41

In [33]:
y_hat = model.predict(X_test)
print(mean_squared_error(y_hat, y_test))
print(sklearn.metrics.r2_score(y_hat, y_test))

0.13696858842145776
0.848129865574843
